In [4]:
import os
import pandas as pd
import requests
from http import HTTPStatus
from bs4 import BeautifulSoup
import re
import requests
from http import HTTPStatus

list_page_url = "https://www.evpost.co.kr/wp/category/실사용기/칼럼/"
start_page = 1    

outputs = []  # 결과를 저장할 리스트

# 상세페이지를 크롤링

def crawl_detail_page(request_url: str):
    try:
        response = requests.get(request_url)
        # HTTP 상태 코드가 200이 아닌 경우
        if response.status_code != HTTPStatus.OK:
            print(f"{response.status_code} 상태 코드가 반환되었습니다. 요청 URL: {request_url}")
            return
        else:
            print(f"{response.status_code} 상태 코드 - 요청이 성공했습니다. 요청 URL: {request_url}")

        # 페이지 HTML 파싱
        html = response.text
        soup = BeautifulSoup(html, "html.parser")

        # 본문 내용이 있는지 확인
        if content := soup.find("div", {"class": "td-post-content"}):
            return content.get_text(strip=True)
        else:
            print("본문 내용을 찾을 수 없습니다.")

    except requests.exceptions.RequestException as e:
        print(f"요청 중 예외가 발생했습니다: {e}")
        return None

# 리스트 페이지를 크롤링
def crawl_list_page(page: int):
    if page == 1:
        request_url = list_page_url
    else:
        request_url = f"{list_page_url}/page/{page}"
    
    response = requests.get(request_url)
    if response.status_code != HTTPStatus.OK:
        print(f"{response.status_code} 상태 코드가 반환되었습니다.")
        return
        
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    
    item_cards = soup.find_all("div", {"class": "td-module-container"})
    for item_card in item_cards:
        entry_title = item_card.find("h3", {"class": "entry-title"})
        if not entry_title:
            continue
        
        # 제목 추출
        title = entry_title.get_text(strip=True)

        # 썸네일 URL 추출
        if image := item_card.find("span", {"class": "entry-thumb"}):
            image = image.get("style")
            image_url = extract_image_url(image)
        else:
            image_url = None

        # 날짜 추출
        if date := item_card.find("time", {"class": "entry-date"}):
            date = date.get_text(strip=True)
        else:
            date = None
        
        # 본문 추출
        ahref = entry_title.find("a").get("href")
        if content := crawl_detail_page(ahref):
            outputs.append({
                "title": title,
                "date": date,
                "image": image_url,
                "content": content,
                "url": ahref
            })
    
    next_btn = soup.find("a", {"aria-label": "next-page"})
    if next_btn:
        next_page = page + 1
        crawl_list_page(next_page)

# 이미지 URL 추출 함수
def extract_image_url(text):
    try:
        return re.search(r"url\((.*?)\)", text).group(1)
    except:
        return None

# 크롤링 실행

# 폴더 경로 지정
folder_path = os.path.join("app_manager", "data")

# 폴더가 없으면 생성
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 파일 경로 지정 및 CSV 저장
file_path = os.path.join(folder_path, "evpost_news.csv")
df = pd.DataFrame(outputs)
df.to_csv(file_path, index=False)

print(f"CSV 파일이 {file_path}에 저장되었습니다.")


200 상태 코드 - 요청이 성공했습니다. 요청 URL: https://www.evpost.co.kr/wp/%ec%ba%90%ec%8a%a4%ed%8d%bc-%ec%9d%bc%eb%a0%89%ed%8a%b8%eb%a6%ad-%eb%94%b13290%eb%a7%8c%ec%9b%90-%ec%a3%bc%ea%b3%a0-%ec%82%ac%ec%84%b8%ec%9a%94/
200 상태 코드 - 요청이 성공했습니다. 요청 URL: https://www.evpost.co.kr/wp/ev3-%ea%b9%a1%ed%86%b5%eb%8f%84-%ed%92%80%ec%98%b5%ec%85%98-%ec%88%98%ec%a4%80%ec%9d%b4%eb%9d%bc%eb%8d%98%eb%8d%b0-feat-%eb%9d%bc%ed%8e%99%ed%84%b0/
200 상태 코드 - 요청이 성공했습니다. 요청 URL: https://www.evpost.co.kr/wp/%ed%85%8c%ec%8a%ac%eb%9d%bc-%ec%97%90%eb%84%88%ec%a7%80-%ec%9d%b4%eb%9e%98%ec%84%9c-%ec%9e%98%eb%90%a0-%ec%88%98%eb%b0%96%ec%97%90-%ec%97%86%ec%8a%b5%eb%8b%88%eb%8b%a4/
200 상태 코드 - 요청이 성공했습니다. 요청 URL: https://www.evpost.co.kr/wp/%ec%9d%b4%eb%9f%ac%eb%a9%b4-%ec%99%84%ec%a0%84%ed%9e%88-%eb%82%98%ea%b0%80%eb%a6%b0%eb%8d%b0/
200 상태 코드 - 요청이 성공했습니다. 요청 URL: https://www.evpost.co.kr/wp/%eb%94%94%ec%9e%90%ec%9d%b8-%eb%a7%9b%ec%a7%91%ec%9d%b4-%eb%a7%90%ec%95%84%ec%a3%bc%eb%8a%94-%ec%a0%84%ea%b8%b0%ec%b0%a8-ft-%ec%95%84%ec%9a%b0%